In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
import numpy as np
from pprint import pprint
from tqdm.auto import tqdm
from tqdm import tqdm_notebook
from scipy.sparse import hstack
from collections import defaultdict, Counter
from ds_tools.ds_tools import CategoricalTransformer
tqdm.pandas()

In [2]:
df_description = pd.read_csv('./data/data_definition.txt', sep='\t')

df_train_genba = pd.read_csv('./data/train_genba.tsv', sep='\t')
df_train_goto = pd.read_csv('./data/train_goto.tsv', sep='\t')

df_train = df_train_goto.merge(df_train_genba, on='pj_no', how='left')
df_train.drop('id', axis=1, inplace=True)

df_test_genba = pd.read_csv('./data/test_genba.tsv', sep='\t')
df_test_goto = pd.read_csv('./data/test_goto.tsv', sep='\t')

df_test = df_test_goto.merge(df_test_genba, on='pj_no', how='left')
test_surface = df_test['tc_mseki']
df_test.drop('id', axis=1, inplace=True)

## Preprocessing

In [3]:
def fill_city_name(name):
    if '市' not in name and '郡' not in name:
        name = '市' + name
    return name

def split_address(df):
    df['jukyo'] = df['jukyo'].str.slice(start=3).str.replace(r'[ヶｹ]', 'ケ')
    df['jukyo'] = df['jukyo'].apply(fill_city_name)
    city_split = df['jukyo'].str.split(r'[市郡]', n=1, expand=True)
    df['city'] = city_split[0]
    street_split = city_split[1].str.split(r'[町区]', n=1, expand=True)
    df['street'] = street_split[0]
    df['address_detail'] = street_split[1].str.strip().replace('', None)
    return df

df_train = split_address(df_train)
df_test = split_address(df_test)

df_train.drop(['kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)
df_test.drop(['kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)

In [4]:
def combine(row, combine_list, tup):
    l = set()
    for col in tup:
        if pd.notnull(row[col]):
            l.add(row[col])
    combine_list.append(','.join(l))


combine_cols = [('yoto', 100), ('road_hk', 100)]
for i, tup in enumerate([['yoto1', 'yoto2'], ['road1_hk', 'road2_hk', 'road3_hk', 'road4_hk']]):
    combine_train = []
    combine_test = []
    
    combine_col_name = combine_cols[i][0]
    _ = df_train.apply(lambda row: combine(row, combine_train, tup), axis=1)
    _ = df_test.apply(lambda row: combine(row, combine_test, tup), axis=1)

    count_vectorizer = CountVectorizer(min_df=combine_cols[i][1])
    combine_train_matrix = count_vectorizer.fit_transform(combine_train).todense()
    combine_test_matrix = count_vectorizer.transform(combine_test).todense()
    for i in range(combine_train_matrix.shape[1]):
        df_train['%s_%d' % (combine_col_name, i)] = combine_train_matrix[:, i]
        df_test['%s_%d' % (combine_col_name, i)] = combine_test_matrix[:, i]
    for col in tup:
        df_train.drop(col, axis=1, inplace=True)
        df_test.drop(col, axis=1, inplace=True)

In [5]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
price_stats = []
for train_idx, valid_idx in splitter.split(df_train):
    price_stats_by_city = defaultdict(dict)
    for city, group in df_train.iloc[train_idx].groupby('city'):
        price_list = group['keiyaku_pr']/group['tc_mseki']
        price_stats_by_city[city]['price_by_city_mean'] = price_list.mean()
        price_stats_by_city[city]['price_by_city_median'] = price_list.median()
        price_stats_by_city[city]['price_by_city_min'] = price_list.min()
        price_stats_by_city[city]['price_by_city_max'] = price_list.max()
        price_stats_by_city[city]['price_by_city_std'] = price_list.std()
        price_stats_by_city[city]['price_by_city_count'] = len(price_list)
    for i, city in enumerate(df_train.iloc[valid_idx]['city']):
        price_stats.append((valid_idx[i], price_stats_by_city[city]))

price_stats_test = [] 
price_stats_by_city = defaultdict(dict)
for city, group in df_train.groupby('city'):
    price_list = group['keiyaku_pr']/group['tc_mseki']
    price_stats_by_city[city]['price_by_city_mean'] = price_list.mean()
    price_stats_by_city[city]['price_by_city_median'] = price_list.median()
    price_stats_by_city[city]['price_by_city_min'] = price_list.min()
    price_stats_by_city[city]['price_by_city_max'] = price_list.max()
    price_stats_by_city[city]['price_by_city_std'] = price_list.std()
    price_stats_by_city[city]['price_by_city_count'] = len(price_list)
for city in df_test['city']:
    price_stats_test.append(price_stats_by_city[city])
    
df_price_stats = pd.DataFrame([x[1] for x in sorted(price_stats, key=lambda x: x[0])])
df_price_stats_test = pd.DataFrame(price_stats_test)

df_train = pd.concat([df_train, df_price_stats], axis=1)
df_test = pd.concat([df_test, df_price_stats_test], axis=1)

for col in ['mseki_rd_hb', 'road3_fi', 'rosenka_hb', 'kempei2', 'road2_mg', 'kaoku_hb', 'bus_hon']:
    df_train[col].replace(0.0, np.nan, inplace=True)
    df_test[col].replace(0.0, np.nan, inplace=True)

In [6]:
df_train['station_name_prefix'] = df_train['rosen_nm1'].str.slice(stop=2)
df_test['station_name_prefix'] = df_test['rosen_nm1'].str.slice(stop=2)

## Train

In [7]:
continue_features = list(df_description[(df_description['データ型'] == '数値') & (df_description['項目名'] != 'pj_no')]['項目名'])
continue_features += ['price_by_city_mean', 'price_by_city_median', 'price_by_city_min', 'price_by_city_max', 
                      'price_by_city_std', 'price_by_city_count']
objective = 'keiyaku_pr'
categorical_features = list(df_train)

for f in continue_features+[objective]:
    if f in categorical_features:
        categorical_features.remove(f)
        
for col in categorical_features:
    if col not in ['pj_no']:
        ct = CategoricalTransformer(min_freq=3)
        df_train[col] = ct.fit_transform(df_train[col])
        df_test[col] = ct.transform(df_test[col])
        
for col in continue_features:
    if col != 'keiyaku_pr':
        scaler = MinMaxScaler()
        df_train[col] = scaler.fit_transform(df_train[col].values.reshape(-1, 1))
        df_test[col] = scaler.transform(df_test[col].values.reshape(-1, 1))
    
df_test['keiyaku_pr'] = 0

## LGB

In [12]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    X_train, y_train = train.drop('keiyaku_pr', axis=1), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = valid.drop('keiyaku_pr', axis=1), np.log(valid['keiyaku_pr']+1)
    regressor = lgb.LGBMRegressor(n_estimators=20000, learning_rate=0.01, silent=False, random_state=28,
                                 bagging_fraction=0.9, bagging_freq=1, feature_fraction=0.8, lambda_l1=0.01)
    regressor.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=500,
                  categorical_feature=categorical_features)
    prediction_list.append(regressor.predict(df_test[df_train.drop(objective, axis=1).columns]))
    best_scores.append(regressor.best_score_['valid_0']['l2'])

print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission.tsv', sep='\t', header=None, index=False)

# 0.01210079

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['address_detail', 'bas_toho1', 'bas_toho2', 'bastei_nm1', 'bastei_nm2', 'bokachiiki', 'bus_yohi', 'chikukeikaku', 'city', 'eki_nm1', 'eki_nm2', 'fi3m_yohi', 'fi4m_yohi', 'gas', 'gesui', 'hiatari', 'hokakisei1', 'hokakisei2', 'hokakisei3', 'hokakisei4', 'hokakyoka', 'hw_status', 'jigata', 'josui', 'jukyo', 'kaihatsukyoka', 'kborjs', 'keikakuroad', 'kinshijiko', 'kobetsu1', 'kobetsu2', 'kobetsu3', 'kobetsu4', 'kodochiku', 'levelplan', 'pj_no', 'road1_sb', 'road2_sb', 'road3_sb', 'road4_sb', 'road_hk_0', 'road_hk_1', 'road_hk_2', 'road_hk_3', 'road_st', 'rosen_nm1', 'rosen_nm2', 'rs_e_kdate2', 'rs_e_kdate3', 'rs_e_m_ari', 'rs_e_m_nashi', 'rs_e_parking', 'rs_e_tahata', 'rs_e_zoki', 'rs_n_kdate2', 'rs_n_kdate3', 'rs_n_m_ari', 'rs_n_m_nashi', 'rs_n_parking', 'rs_n_tahata', 'rs_n_zoki', 'rs_s_kdate2', 'rs_s_kdate3', 'rs_s_m_ari', 'rs_s_m_nashi',

[1]	valid_0's l2: 0.0676443
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's l2: 0.0668841
[3]	valid_0's l2: 0.0661628
[4]	valid_0's l2: 0.0654581
[5]	valid_0's l2: 0.0647353
[6]	valid_0's l2: 0.0640171
[7]	valid_0's l2: 0.0633795
[8]	valid_0's l2: 0.0627105
[9]	valid_0's l2: 0.0620584
[10]	valid_0's l2: 0.0614287
[11]	valid_0's l2: 0.0608165
[12]	valid_0's l2: 0.0601967
[13]	valid_0's l2: 0.059585
[14]	valid_0's l2: 0.0589863
[15]	valid_0's l2: 0.0583881
[16]	valid_0's l2: 0.0578162
[17]	valid_0's l2: 0.0572743
[18]	valid_0's l2: 0.0567199
[19]	valid_0's l2: 0.0561826
[20]	valid_0's l2: 0.0556474
[21]	valid_0's l2: 0.0551091
[22]	valid_0's l2: 0.0545799
[23]	valid_0's l2: 0.054051
[24]	valid_0's l2: 0.0535302
[25]	valid_0's l2: 0.0530232
[26]	valid_0's l2: 0.0525253
[27]	valid_0's l2: 0.0520371
[28]	valid_0's l2: 0.0515457
[29]	valid_0's l2: 0.0510869
[30]	valid_0's l2: 0.050633
[31]	valid_0's l2: 0.0501872
[32]	valid_0's l2: 0.0497484
[33]	valid_0's l2: 0.

[298]	valid_0's l2: 0.0206837
[299]	valid_0's l2: 0.0206727
[300]	valid_0's l2: 0.0206563
[301]	valid_0's l2: 0.020634
[302]	valid_0's l2: 0.0206163
[303]	valid_0's l2: 0.020596
[304]	valid_0's l2: 0.0205791
[305]	valid_0's l2: 0.0205613
[306]	valid_0's l2: 0.0205438
[307]	valid_0's l2: 0.0205276
[308]	valid_0's l2: 0.0205122
[309]	valid_0's l2: 0.0204932
[310]	valid_0's l2: 0.020476
[311]	valid_0's l2: 0.0204574
[312]	valid_0's l2: 0.0204366
[313]	valid_0's l2: 0.0204157
[314]	valid_0's l2: 0.0204002
[315]	valid_0's l2: 0.0203837
[316]	valid_0's l2: 0.0203676
[317]	valid_0's l2: 0.0203513
[318]	valid_0's l2: 0.020335
[319]	valid_0's l2: 0.0203151
[320]	valid_0's l2: 0.0203035
[321]	valid_0's l2: 0.0202861
[322]	valid_0's l2: 0.0202683
[323]	valid_0's l2: 0.0202475
[324]	valid_0's l2: 0.0202316
[325]	valid_0's l2: 0.0202163
[326]	valid_0's l2: 0.0202055
[327]	valid_0's l2: 0.0201927
[328]	valid_0's l2: 0.020176
[329]	valid_0's l2: 0.0201617
[330]	valid_0's l2: 0.0201457
[331]	valid_0's

[600]	valid_0's l2: 0.0180873
[601]	valid_0's l2: 0.0180844
[602]	valid_0's l2: 0.0180812
[603]	valid_0's l2: 0.0180731
[604]	valid_0's l2: 0.0180679
[605]	valid_0's l2: 0.0180642
[606]	valid_0's l2: 0.0180606
[607]	valid_0's l2: 0.0180569
[608]	valid_0's l2: 0.0180539
[609]	valid_0's l2: 0.0180482
[610]	valid_0's l2: 0.0180433
[611]	valid_0's l2: 0.0180409
[612]	valid_0's l2: 0.01804
[613]	valid_0's l2: 0.018035
[614]	valid_0's l2: 0.0180297
[615]	valid_0's l2: 0.0180257
[616]	valid_0's l2: 0.0180245
[617]	valid_0's l2: 0.0180211
[618]	valid_0's l2: 0.0180161
[619]	valid_0's l2: 0.018011
[620]	valid_0's l2: 0.0180098
[621]	valid_0's l2: 0.0180046
[622]	valid_0's l2: 0.018001
[623]	valid_0's l2: 0.0179968
[624]	valid_0's l2: 0.0179925
[625]	valid_0's l2: 0.0179894
[626]	valid_0's l2: 0.0179837
[627]	valid_0's l2: 0.0179817
[628]	valid_0's l2: 0.0179782
[629]	valid_0's l2: 0.0179747
[630]	valid_0's l2: 0.0179678
[631]	valid_0's l2: 0.017965
[632]	valid_0's l2: 0.0179628
[633]	valid_0's 

[906]	valid_0's l2: 0.0174319
[907]	valid_0's l2: 0.0174306
[908]	valid_0's l2: 0.0174306
[909]	valid_0's l2: 0.0174264
[910]	valid_0's l2: 0.0174241
[911]	valid_0's l2: 0.0174224
[912]	valid_0's l2: 0.017421
[913]	valid_0's l2: 0.0174211
[914]	valid_0's l2: 0.0174216
[915]	valid_0's l2: 0.0174179
[916]	valid_0's l2: 0.0174151
[917]	valid_0's l2: 0.017413
[918]	valid_0's l2: 0.0174112
[919]	valid_0's l2: 0.0174109
[920]	valid_0's l2: 0.017409
[921]	valid_0's l2: 0.0174091
[922]	valid_0's l2: 0.0174079
[923]	valid_0's l2: 0.0174057
[924]	valid_0's l2: 0.0174059
[925]	valid_0's l2: 0.0174039
[926]	valid_0's l2: 0.0174041
[927]	valid_0's l2: 0.0174031
[928]	valid_0's l2: 0.0174026
[929]	valid_0's l2: 0.0174014
[930]	valid_0's l2: 0.0174014
[931]	valid_0's l2: 0.0173982
[932]	valid_0's l2: 0.0173983
[933]	valid_0's l2: 0.0173983
[934]	valid_0's l2: 0.0173968
[935]	valid_0's l2: 0.0173949
[936]	valid_0's l2: 0.0173958
[937]	valid_0's l2: 0.0173968
[938]	valid_0's l2: 0.0173951
[939]	valid_0

...

[2905]	valid_0's l2: 0.0115515
[2906]	valid_0's l2: 0.0115506
[2907]	valid_0's l2: 0.0115509
[2908]	valid_0's l2: 0.0115502
[2909]	valid_0's l2: 0.0115504
[2910]	valid_0's l2: 0.0115497
[2911]	valid_0's l2: 0.0115492
[2912]	valid_0's l2: 0.0115496
[2913]	valid_0's l2: 0.0115504
[2914]	valid_0's l2: 0.0115505
[2915]	valid_0's l2: 0.0115502
[2916]	valid_0's l2: 0.0115501
[2917]	valid_0's l2: 0.0115505
[2918]	valid_0's l2: 0.0115516
[2919]	valid_0's l2: 0.0115513
[2920]	valid_0's l2: 0.0115517
[2921]	valid_0's l2: 0.0115518
[2922]	valid_0's l2: 0.011552
[2923]	valid_0's l2: 0.0115518
[2924]	valid_0's l2: 0.0115514
[2925]	valid_0's l2: 0.0115517
[2926]	valid_0's l2: 0.0115514
[2927]	valid_0's l2: 0.0115516
[2928]	valid_0's l2: 0.0115522
[2929]	valid_0's l2: 0.0115526
[2930]	valid_0's l2: 0.0115528
[2931]	valid_0's l2: 0.0115525
[2932]	valid_0's l2: 0.0115523
[2933]	valid_0's l2: 0.0115515
[2934]	valid_0's l2: 0.0115518
[2935]	valid_0's l2: 0.0115515
[2936]	valid_0's l2: 0.0115515
[2937]	va

[3191]	valid_0's l2: 0.0115482
[3192]	valid_0's l2: 0.0115482
[3193]	valid_0's l2: 0.0115477
[3194]	valid_0's l2: 0.0115475
[3195]	valid_0's l2: 0.0115471
[3196]	valid_0's l2: 0.0115475
[3197]	valid_0's l2: 0.0115476
[3198]	valid_0's l2: 0.0115473
[3199]	valid_0's l2: 0.0115472
[3200]	valid_0's l2: 0.0115477
[3201]	valid_0's l2: 0.0115475
[3202]	valid_0's l2: 0.0115479
[3203]	valid_0's l2: 0.0115478
[3204]	valid_0's l2: 0.011547
[3205]	valid_0's l2: 0.011547
[3206]	valid_0's l2: 0.011547
[3207]	valid_0's l2: 0.0115471
[3208]	valid_0's l2: 0.0115476
[3209]	valid_0's l2: 0.0115474
[3210]	valid_0's l2: 0.0115475
[3211]	valid_0's l2: 0.0115475
[3212]	valid_0's l2: 0.0115472
[3213]	valid_0's l2: 0.0115475
[3214]	valid_0's l2: 0.0115476
[3215]	valid_0's l2: 0.0115479
[3216]	valid_0's l2: 0.011549
[3217]	valid_0's l2: 0.0115493
[3218]	valid_0's l2: 0.0115501
[3219]	valid_0's l2: 0.0115497
[3220]	valid_0's l2: 0.0115502
[3221]	valid_0's l2: 0.0115504
[3222]	valid_0's l2: 0.0115515
[3223]	valid

[3482]	valid_0's l2: 0.0115479
[3483]	valid_0's l2: 0.0115477
[3484]	valid_0's l2: 0.0115472
[3485]	valid_0's l2: 0.0115465
[3486]	valid_0's l2: 0.0115466
[3487]	valid_0's l2: 0.0115468
[3488]	valid_0's l2: 0.0115472
[3489]	valid_0's l2: 0.0115477
[3490]	valid_0's l2: 0.0115478
[3491]	valid_0's l2: 0.0115473
[3492]	valid_0's l2: 0.0115473
[3493]	valid_0's l2: 0.0115479
[3494]	valid_0's l2: 0.0115476
[3495]	valid_0's l2: 0.0115477
[3496]	valid_0's l2: 0.0115478
[3497]	valid_0's l2: 0.0115476
[3498]	valid_0's l2: 0.0115474
[3499]	valid_0's l2: 0.0115466
[3500]	valid_0's l2: 0.0115466
[3501]	valid_0's l2: 0.011546
[3502]	valid_0's l2: 0.0115453
[3503]	valid_0's l2: 0.0115446
[3504]	valid_0's l2: 0.0115447
[3505]	valid_0's l2: 0.0115448
[3506]	valid_0's l2: 0.0115441
[3507]	valid_0's l2: 0.0115443
[3508]	valid_0's l2: 0.0115444
[3509]	valid_0's l2: 0.011544
[3510]	valid_0's l2: 0.0115444
[3511]	valid_0's l2: 0.0115447
[3512]	valid_0's l2: 0.0115446
[3513]	valid_0's l2: 0.0115445
[3514]	val

[3774]	valid_0's l2: 0.0115453
[3775]	valid_0's l2: 0.011546
[3776]	valid_0's l2: 0.011546
[3777]	valid_0's l2: 0.0115462
[3778]	valid_0's l2: 0.0115459
[3779]	valid_0's l2: 0.0115457
[3780]	valid_0's l2: 0.0115457
[3781]	valid_0's l2: 0.0115447
[3782]	valid_0's l2: 0.0115436
[3783]	valid_0's l2: 0.0115431
[3784]	valid_0's l2: 0.0115432
[3785]	valid_0's l2: 0.0115433
[3786]	valid_0's l2: 0.011544
[3787]	valid_0's l2: 0.011544
[3788]	valid_0's l2: 0.0115435
[3789]	valid_0's l2: 0.0115431
[3790]	valid_0's l2: 0.0115434
[3791]	valid_0's l2: 0.0115431
[3792]	valid_0's l2: 0.0115441
[3793]	valid_0's l2: 0.0115446
[3794]	valid_0's l2: 0.0115454
[3795]	valid_0's l2: 0.0115459
[3796]	valid_0's l2: 0.0115468
[3797]	valid_0's l2: 0.0115465
[3798]	valid_0's l2: 0.0115466
[3799]	valid_0's l2: 0.0115466
[3800]	valid_0's l2: 0.0115464
[3801]	valid_0's l2: 0.0115464
[3802]	valid_0's l2: 0.0115465
[3803]	valid_0's l2: 0.0115462
[3804]	valid_0's l2: 0.0115456
[3805]	valid_0's l2: 0.0115458
[3806]	valid

5-fold cv mean l2 0.01210942
